<center><h1>BOOK RECOMMENDATION</h1></center>

<center><img src="https://wildlife-travel.com/wp-content/uploads/2019/07/book-recommendation.jpg"></center>




This Dashboard is to Recommend Books for users in [this dataset](https://github.com/caserec/Datasets-for-Recommender-Systems/tree/master/Processed%20Datasets/BookCrossing)
or recommend top books if the user isn't in this dataset

In [10]:
import pandas as pd 
book_info = pd.read_csv("items_info.dat", sep='\t',index_col=False)
book_info

Book_ID        ISBN                                         Book-Title  \
0            1  0060973129                               Decision in Normandy   
1            2  0393045218                             The Mummies of Urumchi   
2            3  0425176428  What If?: The World's Foremost Military Histor...   
3            4  0452264464               Beloved (Plume Contemporary Fiction)   
4            5  0609804618  Our Dumb Century: The Onion Presents 100 Years...   
...        ...         ...                                                ...   
17379    17380  0449209202                         Guide to Confident Living    
17380    17381  2209014026                                   Ludwig Feuerbach   
17381    17382  070892722X  To Bring You Joy (Ulverscroft Large Print Series)   
17382    17383  1570031843  Like Unto Like: A Novel (Southern Classics Ser...   
17383    17384  0435272632                  The Grapes of Wrath (Upper Level)   

                              Book-Author Year-Of-Publication  \
0                            Carlo D'Este                1991   
1                           E.J.W. Barber                1999   
2                           Robert Cowley                2000   
3                           Toni Morrison                1994   
4                               The Onion                1999   
...                                   ...                 ...   
17379                Norman Vincent Peale                1985   
17380                           F. Engels                1976   
17381                       Essie Summers                1992   
17382  Katherine Sherwood Bonner McDowell                1997   
17383                          John Milne                1995   

                                Publisher  \
0                         HarperPerennial   
1               W. W. Norton &amp Company   
2                Berkley Publishing Group   
3                                   Plume   
4                      Three Rivers Press   
...                                   ...   
17379                       Fawcett Books   
17380                          La Dispute   
17381             Ulverscroft Large Print   
17382  University of South Carolina Press   
17383                    Delta Systems Co   

                                             Image-URL-S  \
0      http://images.amazon.com/images/P/0060973129.0...   
1      http://images.amazon.com/images/P/0393045218.0...   
2      http://images.amazon.com/images/P/0425176428.0...   
3      http://images.amazon.com/images/P/0452264464.0...   
4      http://images.amazon.com/images/P/0609804618.0...   
...                                                  ...   
17379  http://images.amazon.com/images/P/0449209202.0...   
17380  http://images.amazon.com/images/P/2209014026.0...   
17381  http://images.amazon.com/images/P/070892722X.0...   
17382  http://images.amazon.com/images/P/1570031843.0...   
17383  http://images.amazon.com/images/P/0435272632.0...   

                                             Image-URL-M  \
0      http://images.amazon.com/images/P/0060973129.0...   
1      http://images.amazon.com/images/P/0393045218.0...   
2      http://images.amazon.com/images/P/0425176428.0...   
3      http://images.amazon.com/images/P/0452264464.0...   
4      http://images.amazon.com/images/P/0609804618.0...   
...                                                  ...   
17379  http://images.amazon.com/images/P/0449209202.0...   
17380  http://images.amazon.com/images/P/2209014026.0...   
17381  http://images.amazon.com/images/P/070892722X.0...   
17382  http://images.amazon.com/images/P/1570031843.0...   
17383  http://images.amazon.com/images/P/0435272632.0...   

                                             Image-URL-L  
0      http://images.amazon.com/images/P/0060973129.0...  
1      http://images.amazon.com/images/P/0393045218.0...  
2      http://images.amazon.com/images/P/0425176428.0...  
3      http://images.amazon.com/im

In [11]:
book_ratings = pd.read_csv("book_ratings.dat", sep='\t',index_col=False)
book_ratings.head()

user  item  rating
0     1  6264     7.0
1     1  4350     7.0
2     1  6252     5.0
3     1   202     9.0
4     1  6266     6.0

In [12]:
book_ratings.describe()

user          item        rating
count  62656.000000  62656.000000  62656.000000
mean    1458.705056   6106.118839      7.953700
std      848.564252   4847.444402      1.717846
min        1.000000      1.000000      1.000000
25%      760.000000   1888.000000      7.000000
50%     1398.000000   4977.000000      8.000000
75%     2218.000000   9538.000000      9.000000
max     2945.000000  17384.000000     10.000000

# Build the model 

In [13]:
from surprise import Dataset, KNNWithMeans, Reader

In [14]:
reader = Reader(rating_scale=(1, 10))

In [15]:
data = Dataset.load_from_df(book_ratings[['user', "item", "rating"]], reader)

In [16]:
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
model = KNNWithMeans(sim_options=sim_options)

In [17]:
trainingSet = data.build_full_trainset()
print(trainingSet.n_items)
print(trainingSet.n_ratings)
print(trainingSet.n_users)

14684
62656
1295


In [18]:
model.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
book_ratings2=book_ratings.drop(['user'], axis=1)
book_ratings2=book_ratings2.groupby('item').mean()
book_ratings2['Book_ID']=book_ratings2.index
book_ratings2.head()


rating  Book_ID
item                   
1     6.000000        1
2     8.333333        2
3     8.750000        3
4     7.750000        4
5     7.818182        5

In [34]:
#join in ratio from the other table

book_info['rating']= None
book_info = book_info.join(book_ratings2.set_index(["Book_ID"])["rating"].to_frame("ratio2"), on=["Book_ID"])

# take ratio2 first, then the existing ratio value if ratio2 is null
book_info["rating"] = book_info["ratio2"].fillna(book_info["rating"])

# delete the ratio2 column
del book_info["ratio2"]
book_info['average_rating']=book_info['rating']
book_info=book_info.drop(['rating'],axis=1)
book_info.tail()


Book_ID        ISBN                                         Book-Title  \
17379    17380  0449209202                         Guide to Confident Living    
17380    17381  2209014026                                   Ludwig Feuerbach   
17381    17382  070892722X  To Bring You Joy (Ulverscroft Large Print Series)   
17382    17383  1570031843  Like Unto Like: A Novel (Southern Classics Ser...   
17383    17384  0435272632                  The Grapes of Wrath (Upper Level)   

                              Book-Author Year-Of-Publication  \
17379                Norman Vincent Peale                1985   
17380                           F. Engels                1976   
17381                       Essie Summers                1992   
17382  Katherine Sherwood Bonner McDowell                1997   
17383                          John Milne                1995   

                                Publisher  \
17379                       Fawcett Books   
17380                          La Dispute   
17381             Ulverscroft Large Print   
17382  University of South Carolina Press   
17383                    Delta Systems Co   

                                             Image-URL-S  \
17379  http://images.amazon.com/images/P/0449209202.0...   
17380  http://images.amazon.com/images/P/2209014026.0...   
17381  http://images.amazon.com/images/P/070892722X.0...   
17382  http://images.amazon.com/images/P/1570031843.0...   
17383  http://images.amazon.com/images/P/0435272632.0...   

                                             Image-URL-M  \
17379  http://images.amazon.com/images/P/0449209202.0...   
17380  http://images.amazon.com/images/P/2209014026.0...   
17381  http://images.amazon.com/images/P/070892722X.0...   
17382  http://images.amazon.com/images/P/1570031843.0...   
17383  http://images.amazon.com/images/P/0435272632.0...   

                                             Image-URL-L  average_rating  
17379  http://images.amazon.com/images/P/0449209202.0...             NaN  
17380  http://images.amazon.com/images/P/2209014026.0...             NaN  
17381  http://images.amazon.com/images/P/070892722X.0...             NaN  
17382  http://images.amazon.com/images/P/1570031843.0...             NaN  
17383  http://images.amazon.com/images/P/0435272632.0...             6.5

In [35]:
def top_5_recommendations(uid):
    
    books_rated_by_user = book_ratings.loc[book_ratings['user']==uid, "item"].values
    
    
    other_books = book_info.loc[~book_info['Book_ID'].isin(books_rated_by_user)].sort_values(
        by='average_rating', ascending=False)
    
    # if the user exists or not
    if uid not in set(book_ratings['user']):
        return other_books[:5]
    
    other_books['user_rate'] = other_books['Book_ID'].apply(lambda x: model.predict(uid, x).est)

    
    return other_books.sort_values(by='user_rate', ascending=False).drop(['user_rate'],axis=1)

In [37]:
top_5_recommendations(-1)[:5]

Book_ID        ISBN                                     Book-Title  \
14785    14786  0553290231  Father, Son and Co: My Life at IBM and Beyond   
16216    16217  3596151465                                Henry der Held.   
7877      7878  088029700X                       True Irish Ghost Stories   
7849      7850  0446360082                   A Fool for Murder: A Mystery   
7836      7837  044040665X                              Beezus and Ramona   

            Book-Author Year-Of-Publication                 Publisher  \
14785  Thomas J. Watson                1991              Bantam Books   
16216       Roddy Doyle                2001  Fischer (Tb.), Frankfurt   
7877       John Seymour                1992             Marboro Books   
7849      Marian Babson                1991              Warner Books   
7836     Beverly Cleary                1994            Yearling Books   

                                             Image-URL-S  \
14785  http://images.amazon.com/images/P/0553290231.0...   
16216  http://images.amazon.com/images/P/3596151465.0...   
7877   http://images.amazon.com/images/P/088029700X.0...   
7849   http://images.amazon.com/images/P/0446360082.0...   
7836   http://images.amazon.com/images/P/044040665X.0...   

                                             Image-URL-M  \
14785  http://images.amazon.com/images/P/0553290231.0...   
16216  http://images.amazon.com/images/P/3596151465.0...   
7877   http://images.amazon.com/images/P/088029700X.0...   
7849   http://images.amazon.com/images/P/0446360082.0...   
7836   http://images.amazon.com/images/P/044040665X.0...   

                                             Image-URL-L  average_rating  
14785  http://images.amazon.com/images/P/0553290231.0...            10.0  
16216  http://images.amazon.com/images/P/3596151465.0...            10.0  
7877   http://images.amazon.com/images/P/088029700X.0...            10.0  
7849   http://images.amazon.com/images/P/0446360082.0...            10.0  
7836   http://images.amazon.com/images/P/044040665X.0...            10.0